In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
words = open('names.txt','r').read().splitlines()


In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)

In [4]:
import random 
random.seed(42)
random.shuffle(words)
# build the dataset
block_size = 8 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 8]) torch.Size([182625])
torch.Size([22655, 8]) torch.Size([22655])
torch.Size([22866, 8]) torch.Size([22866])


In [5]:
for x,y in zip(Xtr[:20], Ytr[:20]):
  print(''.join(itos[ix.item()] for ix in x), '-->', itos[y.item()])

........ --> y
.......y --> u
......yu --> h
.....yuh --> e
....yuhe --> n
...yuhen --> g
..yuheng --> .
........ --> d
.......d --> i
......di --> o
.....dio --> n
....dion --> d
...diond --> r
..diondr --> e
.diondre --> .
........ --> x
.......x --> a
......xa --> v
.....xav --> i
....xavi --> e


In [53]:
#Create linear
class Linear:
    def __init__(self,fan_in,fan_out,bias=True):
        self.weight = torch.randn(fan_in, fan_out)/ fan_out * 0.5
        self.bias = torch.zeros(fan_out) if bias else None
    def __call__(self,x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out
    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])
    

#Create BachNorm1D
class BatchNorm1D:
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    # parameters (trained with backprop)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
    # buffers (trained with a running 'momentum update')
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    if self.training:
      if x.ndim == 2:
        dim = 0
      elif x.ndim == 3:
        dim = (0,1)
      xmean = x.mean(dim, keepdim=True) # batch mean
      xvar = x.var(dim, keepdim=True) # batch variance
    else:
      xmean = self.running_mean
      xvar = self.running_var
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    # update the buffers
    if self.training:
      with torch.no_grad():
        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
    return self.out
  def parameters(self):
    return [self.beta,self.gamma] # for backpropagation


class Tanh:
  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out
  def parameters(self):
    return []
  

class FlattenConsecutive:
    def __init__(self,n):
        self.n = n
    def __call__(self,x):
        B,T,C = x.shape
        x = x.view(B,T//self.n,C*self.n)
        if x.shape[1] == 1:
            x = x.squeeze(1)
        self.out = x
        return self.out
    def parameters(self):
        return []
    
class Embedding:
  
  def __init__(self, num_embeddings, embedding_dim):
    self.weight = torch.randn((num_embeddings, embedding_dim))
    
  def __call__(self, IX):
    self.out = self.weight[IX]
    return self.out
  
  def parameters(self):
    return [self.weight]
    

class Sequential:
  
  def __init__(self, layers):
    self.layers = layers
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    self.out = x
    return self.out
  
  def parameters(self):
    # get parameters of all layers and stretch them out into one list
    return [p for layer in self.layers for p in layer.parameters()]
    



class Softmax:
    def __call__(self,x):
        x = torch.max(x, dim=-1, keepdim=True)
        z = torch.exp(x)
        sum_z = z.sum()
        self.out = z/sum_z
        return self.out
    



class GLU:
    def __init__(self,chan_in,chan_out,bias=True):
        self.weight = torch.randn(chan_in,chan_out)/(chan_in)*0.5
        self.bias = torch.zeros(chan_out) if bias else None
    def __call__(self,x):
        assert x.ndim >=2, "Input tensor must have at least 2 dimensions"
        linear = x @ self.weight
        if self.bias is not None:
            linear += self.bias
        linear_out, gate = torch.chunk(linear, 2, dim=-1)
        
        gate = torch.sigmoid(gate)
        self.out = torch.tanh(linear_out) * gate
        return self.out
    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])
                

In [54]:
torch.manual_seed(42); #set rng for reproductivity

In [59]:
n_emb = 24 #dimensionality of the character embedding vectors
n_hidden = 128 #Number of neurons in the hidden layer
# C = torch.randn((vocab_size,n_emb))
# model = Sequential([
#   Embedding(vocab_size, n_emb),
#   FlattenConsecutive(2), Linear(n_emb * 2, n_hidden, bias=False), BatchNorm1D(n_hidden), Tanh(),
#   FlattenConsecutive(2), Linear(n_hidden*2, n_hidden, bias=False), BatchNorm1D(n_hidden), Tanh(),
#   FlattenConsecutive(2), Linear(n_hidden*2, n_hidden, bias=False), BatchNorm1D(n_hidden), Tanh(),
#   Linear(n_hidden, vocab_size),
# ])


model = Sequential([
   Embedding(vocab_size, n_emb),
   FlattenConsecutive(2), GLU(n_emb * 2, n_hidden, bias=False), BatchNorm1D(n_hidden), Tanh(),
   FlattenConsecutive(2), GLU(n_hidden*2, n_hidden, bias=False), BatchNorm1D(n_hidden), Tanh(),
   FlattenConsecutive(2), GLU(n_hidden*2, n_hidden, bias=False), BatchNorm1D(n_hidden), Tanh(),
   GLU(n_hidden, vocab_size),
 ])

with torch.no_grad():
  model.layers[-1].weight *= 0.1

parameters = model.parameters()
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

76579


In [61]:
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
    #mini batch
    ix = torch.randint(0,Xtr.shape[0],(batch_size,)) #batch size
    Xb,Yb = Xtr[ix],Ytr[ix]
    #forward pass
    # emb = C[Xb] #embed the characters into vectors
    # x = emb.view(emb.shape[0],-1) #concatenate the vectors
    logits = model(Xb)
    loss = F.cross_entropy(logits,Yb)
    #back pass
    for p in parameters:
        p.grad = None
    loss.backward()

    #update
    # lr = lrs[i]
    lr = 0.1 if 1 < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    #track stats
    if i % 1000 == 0:
        print(f'{i:7d}/{max_steps: 7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

      0/ 200000: 3.2963
   1000/ 200000: 2.1594
   2000/ 200000: 2.4213
   3000/ 200000: 2.0733
   4000/ 200000: 2.3041
   5000/ 200000: 2.1589
   6000/ 200000: 2.1546
   7000/ 200000: 2.1296
   8000/ 200000: 2.4750
   9000/ 200000: 2.0119
  10000/ 200000: 2.0702
  11000/ 200000: 2.2716
  12000/ 200000: 1.9945
  13000/ 200000: 2.4214
  14000/ 200000: 1.9221
  15000/ 200000: 1.9830
  16000/ 200000: 1.9716
  17000/ 200000: 2.1320
  18000/ 200000: 2.2565
  19000/ 200000: 2.2727
  20000/ 200000: 1.9151
  21000/ 200000: 2.3956
  22000/ 200000: 2.5075
  23000/ 200000: 1.7348
  24000/ 200000: 2.1403
  25000/ 200000: 2.1294
  26000/ 200000: 2.1008
  27000/ 200000: 2.2249
  28000/ 200000: 1.9280
  29000/ 200000: 2.3139
  30000/ 200000: 2.1184
  31000/ 200000: 2.2699
  32000/ 200000: 2.4948
  33000/ 200000: 1.9422
  34000/ 200000: 1.7589
  35000/ 200000: 1.7637
  36000/ 200000: 2.0716
  37000/ 200000: 2.1046
  38000/ 200000: 1.9806
  39000/ 200000: 1.4946
  40000/ 200000: 2.1039
  41000/ 200000:

In [62]:
#layers in eval mode
for layer in model.layers:
  layer.training = False

In [63]:
# evaluate train and val loss
@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  logits = model(x)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 1.8405946493148804
val 2.0387485027313232


In [64]:
#model test
for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass
      logits = model(torch.tensor([context]))
      # sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

marcitia.
isidh.
krishik.
jima.
kaizon.
clarce.
dusti.
mckenna.
eide.
keylse.
khaley.
samari.
emelia.
drun.
erisha.
gwellton.
kolvin.
raelann.
jhonestobis.
maryhon.
